In [1]:
import re
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.styles import Alignment
from openpyxl.utils.dataframe import dataframe_to_rows

In [2]:
# Files :-
open_file_from = '65027 XII.txt'
save_file_at = 'Output_Excel.xlsx'
saved_excel_file = save_file_at

In [3]:
# Reading the text file:-
with open(open_file_from) as file:
    reader = file.read()

In [4]:
# Defining the patterns to get 'Date', 'School Code', 'School Name' and Region' :-
date_pattern = r"DATE:- (\d{2}/\d{2}/\d{4})"
school_pattern = r"SCHOOL : - (\d+) (.+)"
region_pattern = r"REGION:\s+(\S+)"

# Finding the matches for 'Date', 'School code', 'School name', and 'Region' :-
date_match = re.search(date_pattern, reader)
school_match = re.search(school_pattern, reader)
region_match = re.search(region_pattern, reader)

# Giving variable name to all the data:-
date = date_match.group(1) if date_match else ""
school_code = school_match.group(1) if school_match else ""
school_name = school_match.group(2) if school_match else ""
region = region_match.group(1) if region_match else ""

# Defining the patterns to get all the outro information :-
total_candidates_pattern = r"TOTAL CANDIDATES\s*:\s*(\d+)"
total_pass_pattern = r"TOTAL PASS\s*:\s*(\d+)"
total_comptt_pattern = r"TOTAL COMPTT\.\s*:\s*(\d+)"
total_essential_repeat_pattern = r"TOTAL ESSENTIAL REPEAT\s*:\s*(\d+)"
total_absent_pattern = r"TOTAL ABSENT\s*:\s*(\d+)"

# Finding the matches of above defined pattern in the file:-
total_candidates_match = re.search(total_candidates_pattern, reader)
total_pass_match = re.search(total_pass_pattern, reader)
total_comptt_match = re.search(total_comptt_pattern, reader)
total_essential_repeat_match = re.search(total_essential_repeat_pattern, reader)
total_absent_match = re.search(total_absent_pattern, reader)

# Giving variable names to all the data :-
total_candidates = total_candidates_match.group(1) if total_candidates_match else ""
total_pass = total_pass_match.group(1) if total_pass_match else ""
total_comptt = total_comptt_match.group(1) if total_comptt_match else ""
total_essential_repeat = total_essential_repeat_match.group(1) if total_essential_repeat_match else ""
total_absent = total_absent_match.group(1) if total_absent_match else ""

In [5]:
# Pattern to match the unwanted text (from the top of the string to the line starting with "SCHOOL")
unwanted_pattern = r'DATE:.*?\n.*?-----.*?\n\nSCHOOL.*?\n'

# Remove unwanted text using re.sub
input_string_cleaned = re.sub(unwanted_pattern, '', reader, flags=re.DOTALL)

# Pattern to extract Roll, Gender, and Name
roll_gender_name_pattern = r"(\d+)\s+(\w)\s+([A-Z ]+)"

# Pattern to extract Subject Codes
subject_codes_pattern = r"(\d{3}\s+){5}(\d{3})?"

# Pattern to extract Result
result_pattern = r"\b(PASS|FAIL|COMP|ABST)\b"

# Pattern to extract Marks and Grades for each subject
marks_grades_pattern = r"(\d{3}|AB)\s+([A-Z]\d?)"

In [6]:
# Initializing an empty list to store each student's data as a dictionary
students_data = []

# Initializing variables to keep track of student information
current_student_info = None
current_student_grades = None

# Spliting the input_string by newline characters
lines = reader.strip().split('\n')

In [7]:
# Defining a Function to process and add student data to students_data list
def add_student_data(roll, gender, name, subject_codes, result, marks_grades):
    marks = []
    grades = []
    for mark_grade_tuple in marks_grades:
        mark, grade = mark_grade_tuple[0], mark_grade_tuple[1]
        if mark.isdigit():
            marks.append(int(mark))
        else:
            marks.append(mark)
        grades.append(grade)

    if len(subject_codes) < 6 and len(marks) < 6 and len(grades) < 6:
        subject_codes.append(np.NaN)
        marks.append(np.NaN)
        grades.append(np.NaN)

    row_data = {
        'Roll': roll,
        'Gender': gender,
        'Name': name.strip(),
        'Sub_1': subject_codes[0],
        'Marks_1': marks[0],
        'grade_1': grades[0],
        'Sub_2': subject_codes[1],
        'Marks_2': marks[1],
        'grade_2': grades[1],
        'Sub_3': subject_codes[2],
        'Marks_3': marks[2],
        'grade_3': grades[2],
        'Sub_4': subject_codes[3],
        'Marks_4': marks[3],
        'grade_4': grades[3],
        'Sub_5': subject_codes[4],
        'Marks_5': marks[4],
        'grade_5': grades[4],
        'Sub_6': subject_codes[5],
        'Marks_6': marks[5],
        'grade_6': grades[5],
        'Result': result
    }
    
    students_data.append(row_data)

In [8]:
# Iterating through each line to process student data
for line in lines:
    line = line.strip()
    # Checking if the line contains Roll, Gender, and Name
    if re.match(roll_gender_name_pattern, line):
        current_student_info = line
    # Checking if the line contains Marks and Grades
    elif re.match(marks_grades_pattern, line):
        current_student_grades = line
        # Extracting Roll, Gender, and Name
        roll, gender, name = re.search(roll_gender_name_pattern, current_student_info).groups()

        # Extracting Subject Codes and Separating individual subject codes
        subject_codes_string = re.search(subject_codes_pattern, current_student_info).group()
        subject_codes = re.findall(r"\d{3}", subject_codes_string)

        # Extracting Result
        result = re.search(result_pattern, current_student_info).group()

        # Extracting Marks and Grades for each subject
        marks_grades = re.findall(marks_grades_pattern, current_student_grades)

        # Adding student data to students_data list
        add_student_data(roll, gender, name, subject_codes, result, marks_grades)

In [9]:
# Creating the final DataFrame using the list of student dictionaries
df = pd.DataFrame(students_data)

# Reseting the index of the DataFrame
df.reset_index(drop=True, inplace=True)
df = df.fillna("")

# Printing the DataFrame
# df

In [10]:
# Function to extract numeric value from a string
def extract_numeric(value):
    numeric_part = re.sub(r'\D', '', str(value))
    return int(numeric_part) if numeric_part.isdigit() else 0

# Apply the function to extract numeric values from Marks columns
df['Marks_1'] = df['Marks_1'].apply(extract_numeric)
df['Marks_2'] = df['Marks_2'].apply(extract_numeric)
df['Marks_3'] = df['Marks_3'].apply(extract_numeric)
df['Marks_4'] = df['Marks_4'].apply(extract_numeric)
df['Marks_5'] = df['Marks_5'].apply(extract_numeric)
df['Marks_6'] = df['Marks_6'].apply(extract_numeric)

# Add a new column 'Total_Marks'
df['Total_Marks'] = df['Marks_1'] + df['Marks_2'] + df['Marks_3'] + df['Marks_4'] + df['Marks_5'] + df['Marks_6']

# Calculating and Summing Up the best five marks for each row and adding a new column 'Total Marks (Best 5)'
df['Total Marks (Best 5)'] = df[['Marks_1', 'Marks_2', 'Marks_3', 'Marks_4', 'Marks_5', 'Marks_6']].apply(lambda row: sum(sorted(row, reverse=True)[:5]), axis=1)

# Calculate percentage and add a new column 'Percentage (%)'
total_possible_marks = 500
df['Percentage (%)'] = (df['Total Marks (Best 5)'] / total_possible_marks) * 100
df['Percentage (%)'] = df['Percentage (%)'].apply(lambda x: round(x, 2))

# Printing the DataFrame
# df

In [11]:
# Extract unique subject codes
subject_codes = df[['Sub_1', 'Sub_2', 'Sub_3', 'Sub_4', 'Sub_5', 'Sub_6']].values.flatten()
subject_codes = np.unique(subject_codes)

# Create a new dataframe
new_columns = ['Roll', 'Gender', 'Name']
for code in subject_codes:
    new_columns.append(code)
    new_columns.append('Grade_' + code)
new_columns.append('Total Marks')                # Add a new column 'Total_Marks'
new_columns.append('Total Marks (Best 5)')         # Add a new column 'Total_Marks (Best 5)'

new_df = pd.DataFrame(columns=new_columns)

# Iterate through rows and populate the new dataframe
for _, row in df.iterrows():
    new_row = [row['Roll'], row['Gender'], row['Name']]
    total_marks = 0
    best_marks = []
    for code in subject_codes:
        matching_index = np.where(row[['Sub_1', 'Sub_2', 'Sub_3', 'Sub_4', 'Sub_5', 'Sub_6']] == code)[0]
        if matching_index.size > 0:
            marks = row['Marks_' + str(matching_index[0] + 1)]
            grade = row['grade_' + str(matching_index[0] + 1)]
            total_marks += marks           # Adding total marks in a list
            best_marks.append(marks)       # Adding total marks (Best 5) in a list
            new_row.append(marks)
            new_row.append(grade)
        else:
            new_row.append(np.NaN)
            new_row.append(np.NaN)

    # Adding all `Total_Marks` and `Total_Marks (Best 5)` in new columns
    best_marks.sort(reverse=True)
    total_marks_best_5 = sum(best_marks[:5])
    new_row.append(total_marks)
    new_row.append(total_marks_best_5)

    new_df.loc[len(new_df)] = new_row

# Calculate percentage and add a new column 'Percentage (%)'
total_possible_marks = 500
new_df['Percentage (%)'] = (new_df['Total Marks (Best 5)'] / total_possible_marks) * 100
new_df['Percentage (%)'] = new_df['Percentage (%)'].apply(lambda x: round(x, 2))

new_df = new_df.fillna('')

# Iterate through columns in the DataFrame to check for empty columns and remove it
for col in new_df.columns:
    if all(new_df[col] == ""):
        new_df.drop(columns=[col], inplace=True)

# Remove columns with empty string as name
columns_to_remove = [col for col in new_df.columns if col == ""]
new_df.drop(columns=columns_to_remove, inplace=True)

# Printing the new_df
new_df

,Roll,Gender,Name,030,Grade_030,041,Grade_041,042,Grade_042,043,...,Grade_301,302,Grade_302,322,Grade_322,802,Grade_802,Total Marks,Total Marks (Best 5),Percentage (%)
0,22614342,M,ABHIMANYU KUMAR,,,51.0,D1,63.0,C1,52.0,...,B2,80.0,B1,,,,,405,354,70.8
1,22614343,F,AKANKSHA PRIYA,,,75.0,B1,79.0,A2,74.0,...,A2,76.0,B2,,,,,484,410,82.0
2,22614344,F,AKANKSHI KUMARI,,,95.0,A1,95.0,A1,95.0,...,B1,,,98.0,A1,,,556,472,94.4
3,22614345,M,AKEE KUMAR,,,55.0,C1,66.0,B2,59.0,...,D1,,,,,,,393,339,67.8
4,22614346,M,AMAN KUMAR,,,53.0,C2,62.0,C1,65.0,...,B2,74.0,B2,,,,,418,365,73.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,22614654,F,VAANI BHANWALA,,,,,68.0,B2,69.0,...,B2,,,,,,,374,374,74.8
313,22614655,F,RASHI SINHA,70.0,B2,,,,,,...,A1,,,,,89.0,A2,447,393,78.6
314,22614656,M,AAYUSH RAJ,,,19.0,E,0.0,E,0.0,...,E,,,,,,,49,49,9.8
315,22614657,M,KRISHI RAJ,,,,,0.0,E,0.0,...,E,,,,,,,30,30,6.0


In [29]:
# Creating an Excel Workbook
workbook = Workbook()
sheet = workbook.active
sheet.title = 'Result'

In [30]:
# Adding additional information (Date, School Code, School Name, Region) to the Excel file
sheet['A1'] = 'Date:-'
sheet['A2'] = 'School Code'
sheet['H1'] = 'Region:'

sheet['B1'] = date
sheet['B2'] = school_code
sheet['I1'] = region

start_column = 'C'
end_column = 'G'
merged_cell = sheet[start_column + '3']
merged_cell.alignment = Alignment(horizontal='center', vertical='center')
sheet.merge_cells(f'{start_column}3:{end_column}3')           # Merging cells for the school_name
sheet[f'{start_column}3'] = school_name


In [31]:
# Writing DataFrame rows to Excel, starting from the specified row
for row in dataframe_to_rows(new_df, index=False, header=True, ):
    sheet.append(row)
sheet.append([' '])

start_row = 4

# Adding Outro Information :-
end = sheet.max_row
sheet['A'+str(end+2)] = 'Total Candidates :- '
sheet['D'+str(end+2)] = 'Total Absent :- '
sheet['A'+str(end+3)] = 'Total Pass :- '
sheet['D'+str(end+3)] = 'Total Comptt. :-'
sheet['A'+str(end+4)] = 'Total Essential Repeat :- '

sheet['B'+str(end+2)] = total_candidates
sheet['E'+str(end+2)] = total_absent
sheet['B'+str(end+3)] = total_pass
sheet['E'+str(end+3)] = total_comptt
sheet['B'+str(end+4)] = total_essential_repeat

In [32]:
# Saving the Excel file
workbook.save(save_file_at)

In [33]:
# Create empty dataframes to store rows
science_df = pd.DataFrame(columns= new_df.columns)
commerce_df = pd.DataFrame(columns= new_df.columns)

# Iterate through rows
for index, row in new_df.iterrows():
    if row['042'] != '':
        science_df = pd.concat([science_df, pd.DataFrame([row])], ignore_index=True)
    elif row['030'] != '' or row['054'] != '' or row['055'] != '':
        commerce_df = pd.concat([commerce_df, pd.DataFrame([row])], ignore_index=True)


In [34]:
for col in science_df.columns:
    if all(science_df[col] == ""):
        science_df.drop(columns=[col], inplace=True)

science_df

,Roll,Gender,Name,041,Grade_041,042,Grade_042,043,Grade_043,044,...,Grade_065,301,Grade_301,302,Grade_302,322,Grade_322,Total Marks,Total Marks (Best 5),Percentage (%)
0,22614342,M,ABHIMANYU KUMAR,51.0,D1,63.0,C1,52.0,D2,,...,,76,B2,80.0,B1,,,405,354,70.8
1,22614343,F,AKANKSHA PRIYA,75.0,B1,79.0,A2,74.0,B1,,...,,86,A2,76.0,B2,,,484,410,82.0
2,22614344,F,AKANKSHI KUMARI,95.0,A1,95.0,A1,95.0,A1,,...,,84,B1,,,98.0,A1,556,472,94.4
3,22614345,M,AKEE KUMAR,55.0,C1,66.0,B2,59.0,C2,,...,C2,54,D1,,,,,393,339,67.8
4,22614346,M,AMAN KUMAR,53.0,C2,62.0,C1,65.0,C1,,...,,79,B2,74.0,B2,,,418,365,73.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,22614557,F,ZOHA AYAZ,,,70.0,B1,66.0,B2,71.0,...,C2,89,A2,,,,,464,398,79.6
216,22614653,M,AADITYA KUMAR,,,63.0,C1,52.0,D2,67.0,...,C2,81,B1,,,,,417,365,73.0
217,22614654,F,VAANI BHANWALA,,,68.0,B2,69.0,B2,63.0,...,,79,B2,,,,,374,374,74.8
218,22614656,M,AAYUSH RAJ,19.0,E,0.0,E,0.0,E,,...,E,0,E,,,,,49,49,9.8


In [35]:
for col in commerce_df.columns:
        if all(commerce_df[col] == ""):
            commerce_df.drop(columns=[col], inplace=True)

commerce_df

,Roll,Gender,Name,030,Grade_030,041,Grade_041,048,Grade_048,054,...,Grade_301,302,Grade_302,322,Grade_322,802,Grade_802,Total Marks,Total Marks (Best 5),Percentage (%)
0,22614558,F,AAKRITI,58.0,C2,54.0,C2,73.0,C1,76.0,...,B2,,,,,,,390,338,67.6
1,22614559,M,ABHINAV SINGH,87.0,A2,53.0,C2,84.0,B1,87.0,...,C1,,,,,,,471,418,83.6
2,22614560,M,ADITYA BHUSHAN,65.0,C1,56.0,C1,76.0,B2,68.0,...,C1,,,,,,,400,344,68.8
3,22614561,M,ADITYA KUMAR,50.0,D1,,,64.0,C2,65.0,...,D1,62.0,D1,,,,,357,307,61.4
4,22614562,M,ADITYA RAJ,53.0,C2,,,79.0,B2,61.0,...,C1,,,,,,,397,346,69.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,22614650,F,TANISHA RAJ,86.0,A2,59.0,C1,97.0,A1,87.0,...,B1,,,,,,,501,442,88.4
93,22614651,F,TRUSHALI CHARAN PAHARI,74.0,B1,54.0,C2,84.0,B1,82.0,...,B2,,,,,,,447,393,78.6
94,22614652,M,VARUN KUMAR,80.0,B1,,,88.0,A2,74.0,...,B2,82.0,B1,,,,,473,402,80.4
95,22614655,F,RASHI SINHA,70.0,B2,,,,,77.0,...,A1,,,,,89.0,A2,447,393,78.6


In [36]:
# Open the Excel file to add new sheets and data
with pd.ExcelWriter(saved_excel_file, engine='openpyxl', mode='a') as writer:
    science_df.to_excel(writer, sheet_name='Science', index=False)
    commerce_df.to_excel(writer, sheet_name='Commerce', index=False)